In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1. Hyperparameters
BATCH_SIZE = 64
EPOCHS = 10  # Increase for better results
NUM_CLASSES = 10  # CIFAR-10 has 10 categories
INPUT_SHAPE = (32, 32, 3)  # Each CIFAR-10 image is 32x32x3

# 2. Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 3. Preprocessing / Normalization
# We can normalize pixel values by dividing by 255.0.
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

# y_train and y_test are shape (n, 1) with integer labels [0..9].
# We'll use sparse_categorical_crossentropy, so no need to one-hot encode.

# 4. Data Augmentation
# We'll do on-the-fly augmentation for the training set using ImageDataGenerator.
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# For validation, we usually just normalize (already done), so we can use a simple generator.
val_datagen = ImageDataGenerator()

# Flow training images in batches of BATCH_SIZE using train_datagen generator
train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=BATCH_SIZE
)

# Flow test images using val_datagen (no augmentation)
val_generator = val_datagen.flow(
    x_test, y_test,
    batch_size=BATCH_SIZE
)

# 5. Build a Simple CNN Model
model = models.Sequential([
    # Convolutional Block 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Block 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Block 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten + Dense Layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Helps reduce overfitting
    layers.Dense(NUM_CLASSES, activation='softmax')  # 10 neurons for 10 classes
])

# 6. Compile the Model
# We use 'sparse_categorical_crossentropy' since labels are integers [0..9].
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# 7. Training
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# 8. Evaluate on the Test Set
test_loss, test_acc = model.evaluate(val_generator, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# 9. Save the Model
model.save("cifar10_cnn.h5")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 132ms/step - accuracy: 0.2491 - loss: 2.0006 - val_accuracy: 0.4706 - val_loss: 1.4276
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 105s 134ms/step - accuracy: 0.4394 - loss: 1.5348 - val_accuracy: 0.5284 - val_loss: 1.2971
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 107s 137ms/step - accuracy: 0.5025 - loss: 1.3718 - val_accuracy: 0.5987 - val_loss: 1.1276
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 137s 131ms/step - accuracy: 0.5438 - loss: 1.2838 - val_accuracy: 0.6183 - val_loss: 1.0752
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 106s 136ms/step - accuracy: 0.5703 - loss: 1.2119 - val_accuracy: 0.6344 - val_loss: 1.0261
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 139s 132ms/step - accuracy: 0.5917 - loss: 1.1625 - val_accuracy: 0.6451 - val_loss: 1.0009
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 104s 132ms/step - accuracy: 0.6091 - loss: 1.1196 - val_accuracy: 0.6764 - val_loss: 0.9225
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 105s 135ms/step - accuracy: 0.6232 -

Test Loss: 0.8547
Test Accuracy: 0.7051


In [2]:
# Making a prediction using the saved model
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1. Load the model
model = tf.keras.models.load_model("cifar10_cnn.h5")

# 2. Class labels for CIFAR-10
cifar10_classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# 3. Path to the new image
image_path = "/content/Merc_11.jpeg"

# 4. Load & preprocess
img = load_img(image_path, target_size=(32, 32))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 5. Predict
predictions = model.predict(img_array)  # shape: (1, 10) because there are 10 classes
predicted_index = np.argmax(predictions[0]) # index of the maximum value in predictions array
predicted_class = cifar10_classes[predicted_index]

print("Raw predictions:", predictions)
print("Predicted class index:", predicted_index)
print("Predicted class label:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Raw predictions: [[9.0082418e-03 6.8865746e-01 1.2414565e-05 6.8225013e-08 1.8211622e-08
  8.6944087e-09 1.7472783e-07 4.1592550e-08 4.0930035e-04 3.0191231e-01]]
Predicted class index: 1
Predicted class label: automobile
